# New York City Taxi Fare Prediction

In this project, we shall work with a past Kaggle Playground competition called the [New York City Taxi Fare Prediction](https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction). A subset of the original dataset has been taken from DataCamp to reduce the data size. The [dataset](https://assets.datacamp.com/production/repositories/4443/datasets/1abe6ab7c7c0e880a2f6febcae946a33a9ef5e31/taxi_train_chapter_4.csv) is located in the `data` directory.

The aim of the competition is to predict the fare amount of a taxi ride in New York City using the given information about the number of passengers and the pickup and drop off locations. The evaluation metric for this competition is the root mean-squared error (RMSE).

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

#### Data Preprocessing

In [2]:
df = pd.read_csv("data/taxi_data.csv", index_col="id")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare_amount        20000 non-null  float64
 1   pickup_datetime    20000 non-null  object 
 2   pickup_longitude   20000 non-null  float64
 3   pickup_latitude    20000 non-null  float64
 4   dropoff_longitude  20000 non-null  float64
 5   dropoff_latitude   20000 non-null  float64
 6   passenger_count    20000 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ MB


Let us look at the summary statistics of the numerical columns.

In [3]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,11.303321,-72.478584,39.921043,-72.497221,39.913606,1.658000
std,9.541637,10.525376,6.678592,10.460530,6.139231,1.283674
min,-3.000000,-74.438233,-74.006893,-84.654241,-74.006377,0.000000
25%,6.000000,-73.992150,40.734706,-73.991224,40.734537,1.000000
50%,8.500000,-73.981711,40.752680,-73.980217,40.753583,1.000000
75%,12.500000,-73.966802,40.767443,-73.963729,40.768135,2.000000
max,180.000000,40.766125,401.083332,40.802437,41.366138,6.000000


We can observe that the minimum `fare_amount` value is negative and the minimum `passenger_count` is 0. Let us remove all the rows in the dataset where the aforementioned columns are non-positive.

In [4]:
df = df[(df["fare_amount"] > 0) & (df["passenger_count"] > 0)]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19921 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare_amount        19921 non-null  float64
 1   pickup_datetime    19921 non-null  object 
 2   pickup_longitude   19921 non-null  float64
 3   pickup_latitude    19921 non-null  float64
 4   dropoff_longitude  19921 non-null  float64
 5   dropoff_latitude   19921 non-null  float64
 6   passenger_count    19921 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ MB


There no missing values in the dataset. The `fare_amount` column is the target variable and the rest of the columns are the features. Let us inspect the top 5 rows of the `pickup_datetime` column whose datatype is `object`.

In [5]:
df["pickup_datetime"].head()

id
0    2009-06-15 17:26:21 UTC
1    2010-01-05 16:52:16 UTC
2    2011-08-18 00:35:00 UTC
3    2012-04-21 04:30:42 UTC
4    2010-03-09 07:51:00 UTC
Name: pickup_datetime, dtype: object

We shall convert the column to a pandas `datetime` object.

In [6]:
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])

We shall now remove outliers in the coordinates of the pickup and dropoff locations.

In [7]:
def get_bounds(x):
    mean = x.mean()
    std = x.std()
    cut_off = 3*std
    return mean-cut_off, mean+cut_off

col_list = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']

for col in col_list:
    lower, upper = get_bounds(df[col])
    df = df[(df[col] > lower) & (df[col] < upper)]

#### Feature Engineering

Let us create new features by extracting details from the `pickup_datetime` column, and then drop the column.

In [8]:
df["hour"] = df["pickup_datetime"].dt.hour
df["day_of_week"] = df["pickup_datetime"].dt.dayofweek
df["month"] = df["pickup_datetime"].dt.month
df["year"] = df["pickup_datetime"].dt.year

df.drop("pickup_datetime", axis=1, inplace=True)

We can calculate the distance between the pickup and drop off location using the [haversine formula](https://en.wikipedia.org/wiki/Haversine_formula).

In [9]:
def haversine_distance(lat_1, long_1, lat_2, long_2):
    # average radius of earth in km.
    r = 6371
    lat_diff = lat_2 - lat_1
    long_diff = long_2 - long_1
    hav = np.square(np.sin(lat_diff/2)) + np.cos(lat_1)*np.cos(lat_2)*np.square(np.sin(long_diff/2))
    return 2*r*np.arcsin(np.sqrt(hav))

# convert from degrees to radians.
df["pickup_latitude"] = np.deg2rad(df["pickup_latitude"])
df["pickup_longitude"] = np.deg2rad(df["pickup_longitude"])
df["dropoff_latitude"] = np.deg2rad(df["dropoff_latitude"])
df["dropoff_longitude"] = np.deg2rad(df["dropoff_longitude"])

df["distance (km)"] = haversine_distance(df["pickup_latitude"], df["pickup_longitude"],
                                         df["dropoff_latitude"], df["dropoff_longitude"])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19508 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare_amount        19508 non-null  float64
 1   pickup_longitude   19508 non-null  float64
 2   pickup_latitude    19508 non-null  float64
 3   dropoff_longitude  19508 non-null  float64
 4   dropoff_latitude   19508 non-null  float64
 5   passenger_count    19508 non-null  int64  
 6   hour               19508 non-null  int32  
 7   day_of_week        19508 non-null  int32  
 8   month              19508 non-null  int32  
 9   year               19508 non-null  int32  
 10  distance (km)      19508 non-null  float64
dtypes: float64(6), int32(4), int64(1)
memory usage: 1.5 MB


All the columns are of numeric type. Let us split the dataset into train and test sets.

In [11]:
# features.
X = df.drop("fare_amount", axis=1)
# target.
y = df["fare_amount"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

print(f"The train dataset has {y_train.size} rows and the test dataset has {y_test.size} rows.\n")

The train dataset has 15606 rows and the test dataset has 3902 rows.



#### Modelling

We shall build a stacked model of a random forest regressor and a gradient boosting regressor. Let us first split the training set into 2 parts:

In [12]:
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.5, random_state=21)

We shall train the base-models on the first part of the training dataset. The predictions made by the base-models are given as input to a meta-model, which then makes the final prediction. Let us now train the base-models.

**Random Forest Regressor**

In [13]:
rf_model = RandomForestRegressor(random_state=21)

kf = KFold(n_splits=5, shuffle=True, random_state=21)

rf_params = {'n_estimators': [100, 250, 500],
             'max_depth': [3, 5, 8]}

# find optimal hyperparameters.
rf_cv = GridSearchCV(estimator=rf_model, param_grid=rf_params, cv=kf, n_jobs=-1,
                     scoring='neg_root_mean_squared_error', refit=True)

rf_cv.fit(X_train1, y_train1)

print(f'The best values found are: {rf_cv.best_params_}')

The best values found are: {'max_depth': 8, 'n_estimators': 100}


**Gradient Boosting Regressor**

In [14]:
gb_model = GradientBoostingRegressor(random_state=21)

kf = KFold(n_splits=5, shuffle=True, random_state=21)

gb_params = {'n_estimators': [50, 100, 200],
             'max_depth': [2, 3, 5],
             'subsample': [0.6, 0.7, 0.8]}

# find optimal hyperparameters.
gb_cv = GridSearchCV(estimator=gb_model, param_grid=gb_params, cv=kf, n_jobs=-1,
                     scoring='neg_root_mean_squared_error', refit=True)

gb_cv.fit(X_train1, y_train1)

print(f'The best values found are: {gb_cv.best_params_}')

The best values found are: {'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}


**Predictions**

Predictions are made by the base-models on the 2nd part of the training set, and we use them as features for making predictions with the meta-model. We have chosen a linear regression model with no intercept as the meta-model.

In [15]:
rf_train2_pred = rf_cv.predict(X_train2)
gb_train2_pred = gb_cv.predict(X_train2)

X_train2_preds = np.column_stack((rf_train2_pred, gb_train2_pred))

# meta model
lr = LinearRegression(fit_intercept=False)

lr.fit(X_train2_preds, y_train2)

cv_score = -np.mean(cross_val_score(lr, X_train2_preds, y_train2, cv=kf, scoring='neg_root_mean_squared_error'))

print(f'5-fold cross-validation score (RMSE) on the second part of the training set: {cv_score:.4f}')

5-fold cross-validation score (RMSE) on the second part of the training set: 3.9073


**Stacked Model Evaluation**

We evaluate the performance of the stacked model on the test set. 

In [16]:
rf_test_pred = rf_cv.predict(X_test)
gb_test_pred = gb_cv.predict(X_test)

X_test_preds = np.column_stack((rf_test_pred, gb_test_pred))

y_pred = lr.predict(X_test_preds)

rmse= np.sqrt(np.mean(np.square(y_pred - y_test)))

print(f'RMSE score on the test set: {rmse:.4f}')

RMSE score on the test set: 4.7746




The stacked model performs really well and the RMSE value on the unseen test data is pretty low.